In [ ]:
import os
import json
import tempfile
import requests
import numpy as np
import tensorflow as tf

**STEPS**

1. Train model and save it to tf format

2. Use tensorflow-model-server, defining the port number, path of saved model and model name

3. Create a json string (for prediction) and make a post request to the API

In [ ]:
!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update

deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0  15248      0 --:--:-- --:--:-- --:--:-- 15248
OK
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:6 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease [3,012 B]
Get:7 https://developer.download.nvidia.com/compute/machine-learning/r

In [ ]:
!apt-get install tensorflow-model-server

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  tensorflow-model-server
0 upgraded, 1 newly installed, 0 to remove and 72 not upgraded.
Need to get 210 MB of archives.
After this operation, 0 B of additional disk space will be used.
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable/tensorflow-model-server amd64 tensorflow-model-server all 2.3.0 [210 MB]
Fetched 210 MB in 3s (72.6 MB/s)
Selecting previously unselected package tensorflow-model-server.
(Reading database ... 144579 files and directories currently installed.)
Preparing to unpack .../tensorflow-model-server_2.3.0_all.deb ...
Unpacking tensorflow-model-server (2.3.0) ...
Setting up tensorflow-model-server (2.3.0) ...


In [ ]:
#Create a dataset for the relationship y = 2x-1

def get_y(x):
  return 2*x - 1

xArr = np.array([], dtype=float)
yArr = np.array([], dtype=float)

for i in range(-1,5):
  x = i
  y = get_y(x)
  xArr = np.append(xArr, x)
  yArr = np.append(yArr, y)

In [ ]:
model = tf.keras.Sequential([
                                    tf.keras.layers.Dense(units=1, input_shape=[1])
])

model.compile(optimizer='sgd', loss='mean_squared_error')
history = model.fit(xArr, yArr, epochs=1000, verbose=0)

In [ ]:
model.predict([30])

array([[58.999504]], dtype=float32)

In [ ]:
#Get the directory used for temporary files
MODEL_DIR = tempfile.gettempdir()

version = 1
export_path = os.path.join(MODEL_DIR, str(version))

#If file has been creaed before, delete that file first so model can be replaced
if os.path.isdir(export_path):
  print('Model already created. Deleting it first...')
  !rm -r {export_path}

model.save(export_path, save_format='tf')

print(f'Exported to {export_path}')
!ls -l {export_path}

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /tmp/1/assets
Exported to /tmp/1
total 48
drwxr-xr-x 2 root root  4096 Sep  8 00:57 assets
-rw-r--r-- 1 root root 38214 Sep  8 00:57 saved_model.pb
drwxr-xr-x 2 root root  4096 Sep  8 00:57 variables


In [ ]:
!saved_model_cli show --dir {export_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['dense_16_input'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: serving_default_dense_16_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_16'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
W0908 00:58:10.537353 139657650677632 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/

In [ ]:
#Set the environmental variable 'MODEL_DIR' to '/tmp'
os.environ["MODEL_DIR"] = MODEL_DIR

In [ ]:
%%bash --bg
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=linear \
  --model_base_path="${MODEL_DIR}" > server.log 2>&1

Starting job # 2 in a separate thread.


In [ ]:
!tail server.log

To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2020-09-08 01:18:06.563458: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:199] Restoring SavedModel bundle.
2020-09-08 01:18:06.575865: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:183] Running initialization op on SavedModel bundle at path: /tmp/1
2020-09-08 01:18:06.577832: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:303] SavedModel load for tags { serve }; Status: success: OK. Took 29176 microseconds.
2020-09-08 01:18:06.578120: I tensorflow_serving/servables/tensorflow/saved_model_warmup_util.cc:59] No warmup data file found at /tmp/1/assets.extra/tf_serving_warmup_requests
2020-09-08 01:18:06.578222: I tensorflow_serving/core/loader_harness.cc:87] Successfully loaded servable version {name: linear version: 1}
2020-09-08 01:18:06.579368: I tensorflow_serving/model_servers/server.cc:367] Running gRPC ModelServer at 0.0.0.0:8500 ...
[warn] get

In [ ]:
pred = np.array([[9.0], [10.0]])
#signature_name will be the name of the signature_def during saved_model_cli show. It will define what inputs are expected
#json.dumps will convert a python object to json string
data = json.dumps({'signature_name':'serving_default', 'instances':pred.tolist()})
print(data)

{"signature_name": "serving_default", "instances": [[9.0], [10.0]]}


In [ ]:
#Make a post request to the REST api on port 8501

headers = {'content-type':'application/json'}
json_response = requests.post('http://localhost:8501/v1/models/linear:predict', data=data, headers=headers)
print(json_response.text)

{
    "predictions": [[16.9998913], [18.9998722]
    ]
}


In [ ]:
json.loads(json_response.text)['predictions']

[[16.9998913], [18.9998722]]